In [1057]:
import pandas as pd
import sqlalchemy as sa
import requests
import json

In [1058]:
appid = '0ee1aeec809c143ac777fe7340414f60'
lat=43.6548
lon= -79.3883
units= 'metric'
limit = 10000
offset = 0

#'https://data.ontario.ca/api/3/action/datastore_search?resource_id={}&limit={}&offset={}'.format(resource_id, limit, offset)

current_weather_api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&units={}&appid={}&limit={}&offset={}'.format(lat, lon, units, appid, limit, offset)
#api_url = 'https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&units=metric&appid=0ee1aeec809c143ac777fe7340414f60'

print(current_weather_api_url)

https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&units=metric&appid=0ee1aeec809c143ac777fe7340414f60&limit=10000&offset=0


In [1059]:
api_response = requests.get(current_weather_api_url)

api_response

<Response [200]>

In [1060]:
data = api_response.json() #KEY FOR ASSIGNMENT - HOW TO GET KEYS
data

{'coord': {'lon': -79.3883, 'lat': 43.6548},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 20.99,
  'feels_like': 21.1,
  'temp_min': 19.65,
  'temp_max': 21.77,
  'pressure': 1009,
  'humidity': 75},
 'visibility': 10000,
 'wind': {'speed': 3.6, 'deg': 200},
 'clouds': {'all': 100},
 'dt': 1691819164,
 'sys': {'type': 1,
  'id': 718,
  'country': 'CA',
  'sunrise': 1691835513,
  'sunset': 1691886413},
 'timezone': -14400,
 'id': 6167863,
 'name': 'Downtown Toronto',
 'cod': 200}

In [1061]:
from datetime import datetime

In [1062]:
data_dict = {
    'timezone': data['timezone'],
    'id': data['id'],
    'name': data['name'],
    'coord_lon': data['coord']['lon'],
    'coord_lat': data['coord']['lat'],
    'cod': data['cod'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'main_temp': data['main']['temp'],
    'main_feels_like': data['main']['feels_like'],
    'main_temp_min': data['main']['temp_min'],
    'main_temp_max': data['main']['temp_max'],
    'main_pressure': data['main']['pressure'],
    'main_humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': pd.to_datetime(data['dt'], unit='s'),
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': pd.to_datetime(data['sys']['sunrise'], unit='s'),
    'sys_sunset': pd.to_datetime(data['sys']['sunset'], unit='s')
}

# Create a DataFrame from the dictionary
df = pd.DataFrame([data_dict])

print(df)


   timezone       id              name  coord_lon  coord_lat  cod   
0    -14400  6167863  Downtown Toronto   -79.3883    43.6548  200  \

  weather_main weather_description weather_icon      base  ...  main_humidity   
0       Clouds     overcast clouds          04n  stations  ...             75  \

   visibility  wind_speed  wind_deg  clouds_all                  dt  sys_id   
0       10000         3.6       200         100 2023-08-12 05:46:04     718  \

   sys_country         sys_sunrise          sys_sunset  
0           CA 2023-08-12 10:18:33 2023-08-13 00:26:53  

[1 rows x 25 columns]


coord_lon = data['coord']['lon']
coord_lat=data['coord']['lat']

weather_main = data['weather'][0]['main']
weather_description = data['weather'][0]['description']
weather_icon = data['weather'][0]['icon']



base = data['base']

main_temp = data['main']['temp']
main_feels_like = data['main']['feels_like']
main_temp_min = data['main']['temp_min']
main_temp_max = data['main']['temp_max']
main_pressure = data['main']['pressure']
main_humidity = data['main']['humidity']

visibility = data['visibility']

wind_speed = data['wind']['speed']
wind_deg = data['wind']['deg']

clouds_all = data['clouds']['all']

dt = data['dt']

sys_id = data['sys']['id']
sys_country = data['sys']['country']
sys_sunrise = data['sys']['sunrise']
sys_sunset = data['sys']['sunset']

timezone = data['timezone']
id_value = data['id']
name = data['name']
cod = data['cod']

data_dict = {
    'timezone': timezone,
    'id': id_value,
    'name': name,
    'coord_lon': coord_lon,
    'coord_lat': coord_lat,
    'cod': cod,
    'weather_main': weather_main,
    'weather_description': weather_description,
    'weather_icon': weather_icon,
    'base': base,
    'main_temp': main_temp,
    'main_feels_like': main_feels_like,
    'main_temp_min': main_temp_min,
    'main_temp_max': main_temp_max,
    'main_pressure': main_pressure,
    'main_humidity': main_humidity,
    'visibility': visibility,
    'wind_speed': wind_speed,
    'wind_deg': wind_deg,
    'clouds_all': clouds_all,
    'dt': dt,
    'sys_id': sys_id,
    'sys_country': sys_country,
    'sys_sunrise': sys_sunrise,
    'sys_sunset': sys_sunset
  
}

data_dict['dt'] = datetime.utcfromtimestamp(data_dict['dt'])

# Create a DataFrame from the dictionary
df = pd.DataFrame([data_dict])

print(df)

In [1063]:
df

,timezone,id,name,coord_lon,coord_lat,cod,weather_main,weather_description,weather_icon,base,...,main_humidity,visibility,wind_speed,wind_deg,clouds_all,dt,sys_id,sys_country,sys_sunrise,sys_sunset
0,-14400,6167863,Downtown Toronto,-79.3883,43.6548,200,Clouds,overcast clouds,04n,stations,...,75,10000,3.6,200,100,2023-08-12 05:46:04,718,CA,2023-08-12 10:18:33,2023-08-13 00:26:53


In [1064]:
df.head()

,timezone,id,name,coord_lon,coord_lat,cod,weather_main,weather_description,weather_icon,base,...,main_humidity,visibility,wind_speed,wind_deg,clouds_all,dt,sys_id,sys_country,sys_sunrise,sys_sunset
0,-14400,6167863,Downtown Toronto,-79.3883,43.6548,200,Clouds,overcast clouds,04n,stations,...,75,10000,3.6,200,100,2023-08-12 05:46:04,718,CA,2023-08-12 10:18:33,2023-08-13 00:26:53


In [1065]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timezone             1 non-null      int64         
 1   id                   1 non-null      int64         
 2   name                 1 non-null      object        
 3   coord_lon            1 non-null      float64       
 4   coord_lat            1 non-null      float64       
 5   cod                  1 non-null      int64         
 6   weather_main         1 non-null      object        
 7   weather_description  1 non-null      object        
 8   weather_icon         1 non-null      object        
 9   base                 1 non-null      object        
 10  main_temp            1 non-null      float64       
 11  main_feels_like      1 non-null      float64       
 12  main_temp_min        1 non-null      float64       
 13  main_temp_max        1 non-null      fl

In [1066]:
current_weather = pd.DataFrame(df)

print(current_weather)

   timezone       id              name  coord_lon  coord_lat  cod   
0    -14400  6167863  Downtown Toronto   -79.3883    43.6548  200  \

  weather_main weather_description weather_icon      base  ...  main_humidity   
0       Clouds     overcast clouds          04n  stations  ...             75  \

   visibility  wind_speed  wind_deg  clouds_all                  dt  sys_id   
0       10000         3.6       200         100 2023-08-12 05:46:04     718  \

   sys_country         sys_sunrise          sys_sunset  
0           CA 2023-08-12 10:18:33 2023-08-13 00:26:53  

[1 rows x 25 columns]


In [1067]:
db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'      : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'      : '5432',
    'username'  : 'mredshaw',
    'password'  : '2023!Schulich',
    'database'  : 'mredshaw_db'
}

In [1068]:
db_connection_url = sa.engine.URL.create(
    drivername = db_secret['drivername'],
    username   = db_secret['username'],
    password   = db_secret['password'],
    host       = db_secret['host'],
    port       = db_secret['port'],
    database   = db_secret['database']
)

In [1069]:
engine = sa.create_engine(db_connection_url)

In [1070]:
#with engine.connect() as connection:
#    connection.execute('CREATE SCHEMA IF NOT EXISTS uploads')

In [1071]:
current_weather

,timezone,id,name,coord_lon,coord_lat,cod,weather_main,weather_description,weather_icon,base,...,main_humidity,visibility,wind_speed,wind_deg,clouds_all,dt,sys_id,sys_country,sys_sunrise,sys_sunset
0,-14400,6167863,Downtown Toronto,-79.3883,43.6548,200,Clouds,overcast clouds,04n,stations,...,75,10000,3.6,200,100,2023-08-12 05:46:04,718,CA,2023-08-12 10:18:33,2023-08-13 00:26:53


In [1072]:
current_weather.dtypes

timezone                        int64
id                              int64
name                           object
coord_lon                     float64
coord_lat                     float64
cod                             int64
weather_main                   object
weather_description            object
weather_icon                   object
base                           object
main_temp                     float64
main_feels_like               float64
main_temp_min                 float64
main_temp_max                 float64
main_pressure                   int64
main_humidity                   int64
visibility                      int64
wind_speed                    float64
wind_deg                        int64
clouds_all                      int64
dt                     datetime64[ns]
sys_id                          int64
sys_country                    object
sys_sunrise            datetime64[ns]
sys_sunset             datetime64[ns]
dtype: object

In [1073]:
current_weather.to_sql(
    name = 'current_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
    'timezone': sa.types.INTEGER,
    'id': sa.types.INTEGER,
    'name': sa.types.VARCHAR(255),
    'coord_lon': sa.types.FLOAT,
    'coord_lat': sa.types.FLOAT,
    'cod': sa.types.INTEGER,
    'weather_main': sa.types.VARCHAR(255),
    'weather_description': sa.types.VARCHAR(255),
    'weather_icon': sa.types.VARCHAR(255),
    'base': sa.types.VARCHAR(255),
    'main_temp': sa.types.FLOAT,
    'main_feels_like': sa.types.FLOAT,
    'main_temp_min': sa.types.FLOAT,
    'main_temp_max': sa.types.FLOAT,
    'main_pressure': sa.types.INTEGER,
    'main_humidity': sa.types.INTEGER,
    'visibility': sa.types.INTEGER,
    'wind_speed': sa.types.FLOAT,
    'wind_deg': sa.types.INTEGER,
    'clouds_all': sa.types.INTEGER,
    'dt': sa.types.TIMESTAMP,
    'sys_id': sa.types.INTEGER,
    'sys_country': sa.types.VARCHAR(255),
    'sys_sunrise': sa.types.TIMESTAMP,  # Change this to TIMESTAMP
    'sys_sunset': sa.types.TIMESTAMP   # Change this to TIMESTAMP

})

1

In [1074]:
appid = '0ee1aeec809c143ac777fe7340414f60'
lat=43.6548
lon= -79.3883
limit = 10000
offset = 0

forecast_weather_api_url = 'https://api.openweathermap.org/data/2.5/forecast?lat={}&lon={}&appid={}&limit={}&offset={}'.format(lat, lon, appid, limit, offset)
#api_url = 'https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&units=metric&appid=0ee1aeec809c143ac777fe7340414f60'

print(forecast_weather_api_url)

https://api.openweathermap.org/data/2.5/forecast?lat=43.6548&lon=-79.3883&appid=0ee1aeec809c143ac777fe7340414f60&limit=10000&offset=0


In [1075]:
api_response_forecast = requests.get(forecast_weather_api_url)

api_response_forecast

<Response [200]>

In [1076]:
forecast_data = api_response_forecast.json() #KEY FOR ASSIGNMENT - HOW TO GET KEYS
forecast_data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1691820000,
   'main': {'temp': 294.14,
    'feels_like': 294.25,
    'temp_min': 294.07,
    'temp_max': 294.14,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 999,
    'humidity': 75,
    'temp_kf': 0.07},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04n'}],
   'clouds': {'all': 100},
   'wind': {'speed': 2.67, 'deg': 187, 'gust': 4.02},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2023-08-12 06:00:00'},
  {'dt': 1691830800,
   'main': {'temp': 293.65,
    'feels_like': 293.74,
    'temp_min': 292.68,
    'temp_max': 293.65,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 998,
    'humidity': 76,
    'temp_kf': 0.97},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'clouds': {'all': 92},
   'wind': {'speed': 2.62, 'deg': 157, 'gust': 5.04},
   'visibi

In [1077]:
data_list = []

for item in forecast_data['list']:
    dt = pd.to_datetime(item['dt'], unit='s')
    main_temp = item['main']['temp']
    main_feels_like = item['main']['feels_like']
    temp_min = item['main']['temp_min']
    temp_max = item['main']['temp_max']
    pressure = item['main']['pressure']
    humidity = item['main']['humidity']
    
    weather_id = item['weather'][0]['id']
    weather_main = item['weather'][0]['main']
    weather_description = item['weather'][0]['description']
    weather_icon = item['weather'][0]['icon']
    
    data_list.append({
        'dt': dt,
        'main_temp': main_temp,
        'main_feels_like': main_feels_like,
        'temp_min': temp_min,
        'temp_max': temp_max,
        'pressure': pressure,
        'humidity': humidity,
        'weather_id': weather_id,
        'weather_main': weather_main,
        'weather_description': weather_description,
        'weather_icon': weather_icon
    })

forecast_df = pd.DataFrame(data_list)

print(forecast_df)


                    dt  main_temp  main_feels_like  temp_min  temp_max   
0  2023-08-12 06:00:00     294.14           294.25    294.07    294.14  \
1  2023-08-12 09:00:00     293.65           293.74    292.68    293.65   
2  2023-08-12 12:00:00     292.97           293.23    292.39    292.97   
3  2023-08-12 15:00:00     294.68           295.08    294.68    294.68   
4  2023-08-12 18:00:00     294.34           294.84    294.34    294.34   
5  2023-08-12 21:00:00     297.94           298.28    297.94    297.94   
6  2023-08-13 00:00:00     295.96           296.12    295.96    295.96   
7  2023-08-13 03:00:00     293.56           293.67    293.56    293.56   
8  2023-08-13 06:00:00     291.45           291.29    291.45    291.45   
9  2023-08-13 09:00:00     290.13           289.97    290.13    290.13   
10 2023-08-13 12:00:00     291.08           290.94    291.08    291.08   
11 2023-08-13 15:00:00     295.18           294.90    295.18    295.18   
12 2023-08-13 18:00:00     298.44     

In [1078]:
forecast_df.shape

(40, 11)

In [1079]:
forecast_weather = pd.DataFrame(forecast_df)

print(forecast_weather)

                    dt  main_temp  main_feels_like  temp_min  temp_max   
0  2023-08-12 06:00:00     294.14           294.25    294.07    294.14  \
1  2023-08-12 09:00:00     293.65           293.74    292.68    293.65   
2  2023-08-12 12:00:00     292.97           293.23    292.39    292.97   
3  2023-08-12 15:00:00     294.68           295.08    294.68    294.68   
4  2023-08-12 18:00:00     294.34           294.84    294.34    294.34   
5  2023-08-12 21:00:00     297.94           298.28    297.94    297.94   
6  2023-08-13 00:00:00     295.96           296.12    295.96    295.96   
7  2023-08-13 03:00:00     293.56           293.67    293.56    293.56   
8  2023-08-13 06:00:00     291.45           291.29    291.45    291.45   
9  2023-08-13 09:00:00     290.13           289.97    290.13    290.13   
10 2023-08-13 12:00:00     291.08           290.94    291.08    291.08   
11 2023-08-13 15:00:00     295.18           294.90    295.18    295.18   
12 2023-08-13 18:00:00     298.44     

In [1080]:
forecast_weather.dtypes

dt                     datetime64[ns]
main_temp                     float64
main_feels_like               float64
temp_min                      float64
temp_max                      float64
pressure                        int64
humidity                        int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [1081]:
forecast_weather.to_sql(
    name = 'forecast_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
          'dt' : sa.types.TIMESTAMP,
          'main_temp' : sa.types.FLOAT,
          'main_feels_like': sa.types.FLOAT,
          'temp_min': sa.types.FLOAT,
          'temp_max': sa.types.FLOAT,
          'pressure': sa.types.INTEGER,
          'humidity': sa.types.INTEGER,
          'weather_id': sa.types.INTEGER,
          'weather_main': sa.types.VARCHAR(255),
          'weather_description': sa.types.VARCHAR(255),
          'weather_icon': sa.types.VARCHAR(255)
})

40

In [1082]:
key = '617ed86c2036463688f42f2f808227bb'
lat=43.6548
lon= -79.3883
start_date = '2023-01-01:00'
end_date = '2023-08-09:00'
limit = 10000
offset = 0

history_weather_api_url = 'https://api.weatherbit.io/v2.0/history/daily?lat={}&lon={}&key={}&start_date={}&end_date={}&limit={}&offset={}'.format(lat, lon, key, start_date, end_date,limit, offset)
#api_url = 'https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&units=metric&appid=0ee1aeec809c143ac777fe7340414f60'

print(history_weather_api_url)

https://api.weatherbit.io/v2.0/history/daily?lat=43.6548&lon=-79.3883&key=617ed86c2036463688f42f2f808227bb&start_date=2023-01-01:00&end_date=2023-08-09:00&limit=10000&offset=0


In [1083]:
history_api_response = requests.get(history_weather_api_url)

history_api_response

<Response [200]>

In [1084]:
history_data = history_api_response.json() #KEY FOR ASSIGNMENT - HOW TO GET KEYS
history_data

{'city_id': '6167865',
 'city_name': 'Toronto',
 'country_code': 'CA',
 'data': [{'clouds': 83,
   'datetime': '2023-01-01',
   'dewpt': 0.8,
   'dhi': 24,
   'dni': 207,
   'ghi': 83,
   'max_dhi': 86,
   'max_dni': 739,
   'max_ghi': 370,
   'max_temp': 5,
   'max_temp_ts': 1672560000,
   'max_uv': 0.6,
   'max_wind_dir': 248,
   'max_wind_spd': 6.2,
   'max_wind_spd_ts': 1672549200,
   'min_temp': 2.7,
   'min_temp_ts': 1672596000,
   'precip': 4.1,
   'precip_gpm': 4.1,
   'pres': 1000,
   'revision_status': 'final',
   'rh': 79,
   'slp': 1014,
   'snow': 0,
   'snow_depth': None,
   'solar_rad': 14,
   't_dhi': 572,
   't_dni': 4962,
   't_ghi': 1992,
   't_solar_rad': 340,
   'temp': 3.9,
   'ts': 1672549200,
   'wind_dir': 248,
   'wind_gust_spd': 9,
   'wind_spd': 3.3},
  {'clouds': 84,
   'datetime': '2023-01-02',
   'dewpt': 0.4,
   'dhi': 24,
   'dni': 207,
   'ghi': 83,
   'max_dhi': 86,
   'max_dni': 740,
   'max_ghi': 371,
   'max_temp': 5.6,
   'max_temp_ts': 1672686000

In [1085]:
history_data['data']

[{'clouds': 83,
  'datetime': '2023-01-01',
  'dewpt': 0.8,
  'dhi': 24,
  'dni': 207,
  'ghi': 83,
  'max_dhi': 86,
  'max_dni': 739,
  'max_ghi': 370,
  'max_temp': 5,
  'max_temp_ts': 1672560000,
  'max_uv': 0.6,
  'max_wind_dir': 248,
  'max_wind_spd': 6.2,
  'max_wind_spd_ts': 1672549200,
  'min_temp': 2.7,
  'min_temp_ts': 1672596000,
  'precip': 4.1,
  'precip_gpm': 4.1,
  'pres': 1000,
  'revision_status': 'final',
  'rh': 79,
  'slp': 1014,
  'snow': 0,
  'snow_depth': None,
  'solar_rad': 14,
  't_dhi': 572,
  't_dni': 4962,
  't_ghi': 1992,
  't_solar_rad': 340,
  'temp': 3.9,
  'ts': 1672549200,
  'wind_dir': 248,
  'wind_gust_spd': 9,
  'wind_spd': 3.3},
 {'clouds': 84,
  'datetime': '2023-01-02',
  'dewpt': 0.4,
  'dhi': 24,
  'dni': 207,
  'ghi': 83,
  'max_dhi': 86,
  'max_dni': 740,
  'max_ghi': 371,
  'max_temp': 5.6,
  'max_temp_ts': 1672686000,
  'max_uv': 1.3,
  'max_wind_dir': 235,
  'max_wind_spd': 7.2,
  'max_wind_spd_ts': 1672657200,
  'min_temp': 3.5,
  'min_t

In [1086]:
history_weather = pd.DataFrame(history_data['data'])
history_weather

,clouds,datetime,dewpt,dhi,dni,ghi,max_dhi,max_dni,max_ghi,max_temp,...,solar_rad,t_dhi,t_dni,t_ghi,t_solar_rad,temp,ts,wind_dir,wind_gust_spd,wind_spd
0,83,2023-01-01,0.8,24,207,83,86,739,370,5.0,...,14,572,4962,1992,340,3.9,1672549200,248,9.0,3.3
1,84,2023-01-02,0.4,24,207,83,86,740,371,5.6,...,38,574,4975,2004,905,4.6,1672635600,235,8.5,3.5
2,100,2023-01-03,1.1,24,208,84,86,741,373,4.4,...,16,576,4989,2016,383,3.6,1672722000,58,13.7,4.9
3,100,2023-01-04,1.7,24,209,85,86,742,374,4.4,...,8,577,5005,2030,203,3.5,1672808400,61,19.0,5.8
4,95,2023-01-05,1.0,24,209,85,87,743,376,4.8,...,37,580,5022,2044,880,4.0,1672894800,183,9.0,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,20,2023-08-04,16.5,54,432,323,120,909,921,27.5,...,314,1289,10358,7747,7545,22.6,1691121600,26,9.0,1.2
216,44,2023-08-05,13.6,54,430,321,120,909,919,24.2,...,184,1285,10331,7710,4414,21.3,1691208000,96,7.0,1.5
217,52,2023-08-06,13.8,53,429,320,120,909,916,24.5,...,257,1281,10302,7671,6171,20.1,1691294400,137,12.7,3.7
218,98,2023-08-07,19.7,53,428,318,119,908,914,25.2,...,185,1277,10272,7633,4437,22.2,1691380800,155,13.5,3.8


In [1087]:
history_weather.dtypes

clouds               int64
datetime            object
dewpt              float64
dhi                  int64
dni                  int64
ghi                  int64
max_dhi              int64
max_dni              int64
max_ghi              int64
max_temp           float64
max_temp_ts          int64
max_uv             float64
max_wind_dir         int64
max_wind_spd       float64
max_wind_spd_ts      int64
min_temp           float64
min_temp_ts          int64
precip             float64
precip_gpm         float64
pres                 int64
revision_status     object
rh                 float64
slp                  int64
snow               float64
snow_depth         float64
solar_rad            int64
t_dhi                int64
t_dni                int64
t_ghi                int64
t_solar_rad          int64
temp               float64
ts                   int64
wind_dir             int64
wind_gust_spd      float64
wind_spd           float64
dtype: object

In [1088]:
history_weather.to_sql(
    name='history_weather',
    schema='uploads',
    con=engine,
    if_exists='replace',
    index=False,
    method='multi',
    dtype={
        'clouds': sa.types.INTEGER,
        'datetime': sa.types.VARCHAR(255),
        'dewpt': sa.types.FLOAT,
        'dhi': sa.types.INTEGER,
        'dni': sa.types.INTEGER,
        'ghi': sa.types.INTEGER,
        'max_dhi': sa.types.INTEGER,
        'max_dni': sa.types.INTEGER,
        'max_ghi': sa.types.INTEGER,
        'max_temp': sa.types.FLOAT,
        'max_temp_ts': sa.types.INTEGER,
        'max_uv': sa.types.FLOAT,
        'max_wind_dir': sa.types.INTEGER,
        'max_wind_spd': sa.types.FLOAT,
        'max_wind_spd_ts': sa.types.INTEGER,
        'min_temp': sa.types.FLOAT,
        'min_temp_ts': sa.types.INTEGER,
        'precip': sa.types.FLOAT,
        'precip_gpm': sa.types.FLOAT,
        'pres': sa.types.INTEGER,
        'revision_status': sa.types.VARCHAR(255),
        'rh': sa.types.FLOAT,
        'slp': sa.types.INTEGER,
        'snow': sa.types.FLOAT,
        'snow_depth': sa.types.FLOAT,
        'solar_rad': sa.types.INTEGER,
        't_dhi': sa.types.INTEGER,
        't_dni': sa.types.INTEGER,
        't_ghi': sa.types.INTEGER,
        't_solar_rad': sa.types.INTEGER,
        'temp': sa.types.FLOAT,
        'ts': sa.types.INTEGER,
        'wind_dir': sa.types.INTEGER,
        'wind_gust_spd': sa.types.FLOAT,
        'wind_spd': sa.types.FLOAT
    }
)

220